In [26]:
from ipfml import processing
from ipfml import utils
from ipfml import metrics
from PIL import Image
from scipy import signal
from skimage import color
import scipy.stats as stats
import seaborn as sns
import cv2
import numpy as np
import matplotlib.pyplot as plt
import os

from pylab import *
from skimage import data, io, color
from sklearn.decomposition import TruncatedSVD

In [27]:
data_folder = "../fichiersSVD_light"

# SVD Scene analysis on Synthesis Images 

## Utils functions definition

In [28]:
def compute_images_path(dict_data):
    
    all_images_path = []
    for cur_dict in dict_data:
        scene = cur_dict['name']
        prefix = cur_dict['prefix']
        indices = cur_dict['indices']

        scene_images_path = []
        for index in indices:
            path = os.path.join(data_folder, os.path.join(scene, prefix + index + ".png"))
            scene_images_path.append(path)
            
        all_images_path.append(scene_images_path)
            
    return all_images_path

In [29]:
def display_sv_data(dict_data, interval, all_images_path):
    
    sv_values = []
    plt.figure(figsize=(25, 20))
    begin, end = interval
    
    for id_dict, cur_dict in enumerate(dict_data):
        
        scene_name = cur_dict['name']
        image_indices = cur_dict['indices']
        scene_sv_values = []
    
        for id_img, img_path in enumerate(all_images_path[id_dict]):
            img = Image.open(img_path)
            print(img_path)
            
            blocks = processing.divide_in_blocks(img, (200, 200))
            block = np.array(blocks[0])
            
            if block.ndim == 3:
                U, s, V = processing.get_LAB_L_SVD(block)
            else:
                U, s, V = metrics.get_SVD(block)
                
            data = s[begin:end]
            plt.plot(data, label=scene_name + '_' + str(image_indices[id_img]))
            scene_sv_values.append(data)
            
        sv_values.append(scene_sv_values)

    plt.legend(fontsize=18)
    plt.show()
    return sv_values

## Scenes zones data

In [30]:
# start 00020 - ref 00900 - step 10
dict_appart = {'name': 'Appart1opt02', 
               'prefix': 'appartAopt_', 
               'indices': ["00020", "00200", "00900"]}

# start 00050 - ref 01200 - step 10
dict_cuisine = {'name': 'Cuisine01', 
               'prefix': 'cuisine01_', 
               'indices': ["00050", "00400", "01200"]}

# start 00020 - ref 00950 - step 10
dict_sdb_c = {'name': 'SdbCentre', 
               'prefix': 'SdB2_', 
               'indices': ["00020", "00400", "00950"]}

# start 00020 - ref 00950 - step 10
dict_sdb_d = {'name': 'SdbDroite', 
               'prefix': 'SdB2_D_', 
               'indices': ["00020", "00400", "00950"]}

In [31]:
all_dicts = [dict_appart, dict_cuisine, dict_sdb_c, dict_sdb_d]
interval = (30, 200)

../fichiersSVD_light/Appart1opt02/appartAopt_00020.png
../fichiersSVD_light/Appart1opt02/appartAopt_00200.png
../fichiersSVD_light/Appart1opt02/appartAopt_00900.png
../fichiersSVD_light/Cuisine01/cuisine01_00050.png
../fichiersSVD_light/Cuisine01/cuisine01_00400.png
../fichiersSVD_light/Cuisine01/cuisine01_01200.png
../fichiersSVD_light/SdbCentre/SdB2_00020.png
../fichiersSVD_light/SdbCentre/SdB2_00400.png
../fichiersSVD_light/SdbCentre/SdB2_00950.png
../fichiersSVD_light/SdbDroite/SdB2_D_00020.png
../fichiersSVD_light/SdbDroite/SdB2_D_00400.png
../fichiersSVD_light/SdbDroite/SdB2_D_00950.png


In [32]:
all_images_path = compute_images_path(all_dicts)
sv_values = display_sv_data(all_dicts, interval, all_images_path)

### Use of Truncated SVD dimension reduction

In [41]:
all_trunc_svd_sv = []

svd = TruncatedSVD(n_components=30, n_iter=100, random_state=42)    

for scene_path in all_images_path:
    
    scene_trunc_svd_sv = []

    for sample_level_path in scene_path:
    
        current_image = io.imread(sample_level_path)
        current_image = metrics.get_LAB_L(current_image)
        
        print(sample_level_path)
        transformed_image = svd.fit_transform(current_image) 
        restored_image = svd.inverse_transform(transformed_image)
        
        reduced_image = (current_image - restored_image)
        
        U, s, V = metrics.get_SVD(reduced_image)
        scene_trunc_svd_sv.append(s)
        
    all_trunc_svd_sv.append(scene_trunc_svd_sv)

../fichiersSVD_light/Appart1opt02/appartAopt_00020.png
../fichiersSVD_light/Appart1opt02/appartAopt_00200.png
../fichiersSVD_light/Appart1opt02/appartAopt_00900.png
../fichiersSVD_light/Cuisine01/cuisine01_00050.png
../fichiersSVD_light/Cuisine01/cuisine01_00400.png
../fichiersSVD_light/Cuisine01/cuisine01_01200.png
../fichiersSVD_light/SdbCentre/SdB2_00020.png
../fichiersSVD_light/SdbCentre/SdB2_00400.png
../fichiersSVD_light/SdbCentre/SdB2_00950.png
../fichiersSVD_light/SdbDroite/SdB2_D_00020.png
../fichiersSVD_light/SdbDroite/SdB2_D_00400.png
../fichiersSVD_light/SdbDroite/SdB2_D_00950.png


In [44]:
plt.figure(figsize=(25, 20))
interval = (30, 170)
begin, end = interval

for id_dict, cur_dict in enumerate(all_dicts):

    scene_name = cur_dict['name']
    image_indices = cur_dict['indices']

    for id_img, img_path in enumerate(all_images_path[id_dict]):
        
        sv = all_trunc_svd_sv[id_dict][id_img]
        plt.plot(sv[30:200], label=scene_name + '_' + str(image_indices[id_img]))

plt.legend(fontsize=18)
plt.show()